# 1. Popularity, High Rated Based 

In [1]:
ratings={
     'Dave':{'달콤한인생':5,'범죄도시':3,'샤인':3},
     'David':{'달콤한인생':5,'범죄도시':1,'샤인':4},
     'Alex':{'달콤한인생':0,'범죄도시':4,'샤인':5},
     'Andy':{'달콤한인생':2,'범죄도시':1,'샤인':5}
}

In [2]:
movie_dict = dict()
for rating in ratings:
    for movie in ratings[rating].keys():
        if movie not in movie_dict:
            movie_dict[movie] = ratings[rating][movie]
        else:
            movie_dict[movie] = (movie_dict[movie] + ratings[rating][movie]) 

for movie in ratings[rating].keys():
    movie_dict[movie] = movie_dict[movie] / 4

import operator
sorted_x = sorted(movie_dict.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_x[:2])

[('샤인', 4.25), ('달콤한인생', 3.0)]


# 2.2. Collaborative Filtering
- 데이터 구성 : 사용자가 입력한 선호도(평점)를 사용하여 사용자-항목 선호도(평점) 행렬을 만든다.
- 유사도 계산 : 1 단계에서 만들어진 행렬을 사용하여 사용자들 간의 유사도를 계산한다.
- 예측 값 계산 및 추천 목록 생성 : 사용자들 간의 유사도를 바탕으로 모든 항목에 대해 예측 값을 계산하고 높은 예측 값을 갖는 상위 N개의 추천 목록을 생성한다.

In [4]:
ratings.get('David')

{'달콤한인생': 5, '범죄도시': 1, '샤인': 4}

In [5]:
# 둘간의 유사도를 측정해보자(유클라디언거리)

import math
def sim(i, j):
    return math.sqrt(pow(i,2)+pow(j,2))

In [7]:
var1 =  ratings['Alex']['범죄도시'] - ratings['Andy']['범죄도시']
var2 =  ratings['Alex']['샤인'] - ratings['Andy']['샤인']

In [8]:
sim(var1,var2)

3.0

In [9]:
#Alex가 평가한 범죄도시, 샤인을 모두 평가한 사용자와 모두 거리 구하기 (유사도 구하기)

for i in ratings:
    if i!='Alex':
        num1 = ratings.get('Alex').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Alex').get('샤인') - ratings.get(i).get('샤인')
        print(i," : ", sim(num1,num2))

Dave  :  2.23606797749979
David  :  3.1622776601683795
Andy  :  3.0


In [12]:
# 유사도에 대한 normalization
for i in ratings:
    if i!='Alex':
        num1 = ratings.get('Alex').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Alex').get('샤인') - ratings.get(i).get('샤인')
        print(i," : ", 1 / ( 1 + sim(num1,num2) ) )

Dave  :  0.3090169943749474
David  :  0.2402530733520421
Andy  :  0.25


#### Dave가 평가한 범죄도시와 샤인 모두 평가한 사용자와의 거리를 구해서, 가장 Dave와 유사한 사용자 구하기

In [13]:
for i in ratings:
    if i!='Dave':
        num1 = ratings.get('Dave').get('범죄도시') - ratings.get(i).get('범죄도시')
        num2 = ratings.get('Dave').get('샤인') - ratings.get(i).get('샤인')
        print(i," : ", 1 / ( 1 + sim(num1,num2) ) )

David  :  0.3090169943749474
Alex  :  0.3090169943749474
Andy  :  0.2612038749637414


# 유사도 측정

# 3.1. 평균제곱차이 유사도 (Mean Squared Difference Similarity)

### Mean Squared Difference
### msd(u,v)=1|Iuv|⋅∑i∈Iuv(rui−rvi)2

a. Iuv는 사용자 u와 사용자 v 모두에 의해 평가된 상품의 집합 <br>
b. |Iuv|는 사용자 u와 사용자 v 모두에 의해 평가된 상품의 수 <br>
c. Item-based Collaborative Filter 경우: 상품 i와 상품 j간의 msd <br>
d. msd(i,j)=1|Uij|⋅∑u∈Uij(rui−ruj)2 <br>
e. 위 식에서 Uij는 상품 i와 상품 j 모두를 평가한 사용자의 집합이고 |Uij|는 상품 i와 상품 j 모두를 평가한 사용자의 수

In [16]:
def sim_msd(data, name1, name2):
    sum = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum += pow(data[name1][movies]- data[name2][movies], 2)
            count += 1

    return 1 / ( 1 + (sum / count) )

# 3.2. 코사인 유사도 (Cosine Similarity)

코사인 유사도(Cosine Similarity): 두 특성 벡터간의 유사 정도를 코사인 값으로 표현한 것임

Cosine Similarity는 −1에서 1까지의 값을 가지며, −1은 서로 완전히 반대되는 경우, 0은 서로 독립적인 경우, 1은 서로 완전히 같은 경우를 의미함

x⋅y=|x||y|cosθ
cosθ=x⋅y|x||y|

In [17]:
def sim_msd(data, name1, name2):
    sum = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum += pow(data[name1][movies]- data[name2][movies], 2)
            count += 1

    return 1 / ( 1 + (sum / count) )

## Dave와 Andy의cos유사도 구하기

In [18]:
import math
def sim_cosine(data, name1, name2):
    sum_name1 = 0
    sum_name2 = 0
    sum_name1_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies], 2)
            sum_name2 += pow(data[name2][movies], 2)
            sum_name1_name2 += data[name1][movies]*data[name2][movies]
    
    return sum_name1_name2 / (math.sqrt(sum_name1)*math.sqrt(sum_name2))


In [20]:
sim_cosine(ratings,"Dave","Andy")

0.7795844649455863

# 3.3. 피어슨 유사도 (Pearson Similarity)

In [22]:
def sim_pearson(data, name1, name2):
    avg_name1 = 0
    avg_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            avg_name1 = data[name1][movies]
            avg_name2 = data[name2][movies]
            count += 1
    
    avg_name1 = avg_name1 / count
    avg_name2 = avg_name2 / count
    
    sum_name1 = 0
    sum_name2 = 0
    sum_name1_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies] - avg_name1, 2)
            sum_name2 += pow(data[name2][movies] - avg_name2, 2)
            sum_name1_name2 += (data[name1][movies] - avg_name1) * (data[name2][movies] - avg_name2)
    
    return sum_name1_name2 / (math.sqrt(sum_name1)*math.sqrt(sum_name2))

In [25]:
sim_pearson(ratings, 'Dave','Alex')

0.21664456120656048

In [26]:
def top_match(data, name, index=3, sim_function=sim_pearson):
    li=[]
    for i in data: #딕셔너리를 돌고
        if name!=i: #자기 자신이 아닐때만
            li.append((sim_function(data,name,i),i)) #sim_function()을 통해 상관계수를 구하고 li[]에 추가
    li.sort() #오름차순
    li.reverse() #내림차순
    return li[:index]

In [27]:
top_match(ratings, 'Dave', 3)

[(0.868092293129412, 'David'),
 (0.39840953644479793, 'Andy'),
 (0.21664456120656048, 'Alex')]

# top_match를 사용해서 DAVE와 가장 유사한 사용자는?

In [28]:
top_match(ratings, 'Dave', 3, sim_function=sim_msd)

[(0.37499999999999994, 'David'), (0.15, 'Andy'), (0.09090909090909091, 'Alex')]

In [29]:
top_match(ratings, 'Dave', 3, sim_function=sim_cosine)

[(0.9412416106700233, 'David'),
 (0.7795844649455863, 'Andy'),
 (0.6430394361098802, 'Alex')]

In [30]:
top_match(ratings, 'Dave', 3, sim_function=sim_pearson)

[(0.868092293129412, 'David'),
 (0.39840953644479793, 'Andy'),
 (0.21664456120656048, 'Alex')]

## 데이터 분석(추천시스템) - 유사도와 KNN을 활용한 예측 값 계산 및 추천 목록 생성 기법

#### 유사도와 KNN을 활용한 예측 값 계산 및 추천 목록 생성 기법

##### K Nearest Neighbors(KNN) 가중치 예측 기법

: 유사도가 구해지면 평점을 예측하고자 하는 사용자(또는 상품)와 유사도가 큰 k 개의 사용자(또는 상품) 벡터를 사용하여 가중 평균을 구해서 가중치를 예측<br>

##### KNNBasic<br>
평점들을 단순히 가중 평균한다. 다음 식에서 Nk는 k개의 가장 유사도가 큰 벡터의 집합이다.

In [31]:
ratings_expand = {
    '마동석': {
        '택시운전사': 3.5,
        '남한산성': 1.5,
        '킹스맨:골든서클': 3.0,
        '범죄도시': 3.5,
        '아이 캔 스피크': 2.5,
        '꾼': 3.0,
    },
    '이정재': {
        '택시운전사': 5.0,
        '남한산성': 4.5,
        '킹스맨:골든서클': 0.5,
        '범죄도시': 1.5,
        '아이 캔 스피크': 4.5,
        '꾼': 5.0,
    },
    '윤계상': {
        '택시운전사': 3.0,
        '남한산성': 2.5,
        '킹스맨:골든서클': 1.5,
        '범죄도시': 3.0,
        '꾼': 3.0,
        '아이 캔 스피크': 3.5,
    },
    '설경구': {
        '택시운전사': 2.5,
        '남한산성': 3.0,
        '범죄도시': 4.5,
        '꾼': 4.0,
    },
    '최홍만': {
        '남한산성': 4.5,
        '킹스맨:골든서클': 3.0,
        '꾼': 4.5,
        '범죄도시': 3.0,
        '아이 캔 스피크': 2.5,
    },
    '홍수환': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '킹스맨:골든서클': 1.0,
        '범죄도시': 3.0,
        '꾼': 3.5,
        '아이 캔 스피크': 2.0,
    },
    '나원탁': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '꾼': 3.0,
        '범죄도시': 5.0,
        '아이 캔 스피크': 3.5,
    },
    '소이현': {
        '남한산성': 4.5, 
        '아이 캔 스피크': 1.0,
        '범죄도시': 4.0
    }
}

In [32]:
def getRecommendation (data, person, k=3, sim_function=sim_pearson):
    
    result = top_match(data, person, k)
    
    score = 0 # 평점 합을 위한 변수
    li = list() # 리턴을 위한 리스트
    score_dic = dict() # 유사도 총합을 위한 dic
    sim_dic = dict() # 평점 총합을 위한 dic

    for sim, name in result: # 튜플이므로 한번에
        print(sim, name)
        if sim < 0 : continue #유사도가 양수인 사람만
        for movie in data[name]: 
            if movie not in data[person]: #name이 평가를 내리지 않은 영화
                score += sim * data[name][movie] # 그사람의 영화평점 * 유사도
                score_dic.setdefault(movie, 0) # 기본값 설정
                score_dic[movie] += score # 합계 구함

                # 조건에 맞는 사람의 유사도의 누적합을 구한다
                sim_dic.setdefault(movie, 0) 
                sim_dic[movie] += sim

            score = 0  #영화가 바뀌었으니 초기화한다
    
    for key in score_dic: 
        score_dic[key] = score_dic[key] / sim_dic[key] # 평점 총합/ 유사도 총합
        li.append((score_dic[key],key)) # list((tuple))의 리턴을 위해서.
    li.sort() #오름차순
    li.reverse() #내림차순
    return li

In [33]:
getRecommendation(ratings_expand, '소이현')

0.9330597055272909 홍수환
0.8909876971472571 최홍만
0.8452277090445156 나원탁


[(3.675468553454334, '꾼'),
 (3.0000000000000004, '택시운전사'),
 (1.976934805357391, '킹스맨:골든서클')]

* getRecommendation를 사용해서 최홍만 과 가장 유사한 사용자는? 
* 단 유사도 함수를 sim_cosine, k는 2를 사용하시오

In [34]:
getRecommendation(ratings_expand, '최홍만', k=2, sim_function=sim_cosine)

0.9608329054174726 홍수환
0.9517663735117331 소이현


[(3.0, '택시운전사')]

### 4.2. KNNWithMeans¶

평점들을 평균값 기준으로 가중 평균한다.

In [35]:
for name in ratings_expand:
    sum = 0
    count = 0
    for movies in ratings_expand[name]:
        sum += ratings_expand[name][movies]
        count += 1
    ratings_expand[name]['avg'] = sum / count

In [37]:
ratings_expand

{'마동석': {'택시운전사': 3.5,
  '남한산성': 1.5,
  '킹스맨:골든서클': 3.0,
  '범죄도시': 3.5,
  '아이 캔 스피크': 2.5,
  '꾼': 3.0,
  'avg': 2.8333333333333335},
 '이정재': {'택시운전사': 5.0,
  '남한산성': 4.5,
  '킹스맨:골든서클': 0.5,
  '범죄도시': 1.5,
  '아이 캔 스피크': 4.5,
  '꾼': 5.0,
  'avg': 3.5},
 '윤계상': {'택시운전사': 3.0,
  '남한산성': 2.5,
  '킹스맨:골든서클': 1.5,
  '범죄도시': 3.0,
  '꾼': 3.0,
  '아이 캔 스피크': 3.5,
  'avg': 2.75},
 '설경구': {'택시운전사': 2.5, '남한산성': 3.0, '범죄도시': 4.5, '꾼': 4.0, 'avg': 3.5},
 '최홍만': {'남한산성': 4.5,
  '킹스맨:골든서클': 3.0,
  '꾼': 4.5,
  '범죄도시': 3.0,
  '아이 캔 스피크': 2.5,
  'avg': 3.5},
 '홍수환': {'택시운전사': 3.0,
  '남한산성': 4.0,
  '킹스맨:골든서클': 1.0,
  '범죄도시': 3.0,
  '꾼': 3.5,
  '아이 캔 스피크': 2.0,
  'avg': 2.75},
 '나원탁': {'택시운전사': 3.0,
  '남한산성': 4.0,
  '꾼': 3.0,
  '범죄도시': 5.0,
  '아이 캔 스피크': 3.5,
  'avg': 3.7},
 '소이현': {'남한산성': 4.5, '아이 캔 스피크': 1.0, '범죄도시': 4.0, 'avg': 3.1666666666666665}}

In [38]:
def getRecommendation (data, person, k=3, sim_function=sim_pearson):
    
    result = top_match(data, person, k)
    
    score = 0 # 평점 합을 위한 변수
    li = list() # 리턴을 위한 리스트
    score_dic = dict() # 유사도 총합을 위한 dic
    sim_dic = dict() # 평점 총합을 위한 dic

    for sim, name in result: # 튜플이므로 한번에
        print(sim, name)
        if sim < 0 : continue #유사도가 양수인 사람만
        for movie in data[name]: 
            if movie not in data[person]: #name이 평가를 내리지 않은 영화
                score += sim * (data[name][movie] - data[name]['avg']) # 그사람의 영화평점 * 유사도
                score_dic.setdefault(movie, 0) # 기본값 설정
                score_dic[movie] += score # 합계 구함

                # 조건에 맞는 사람의 유사도의 누적합을 구한다
                sim_dic.setdefault(movie, 0) 
                sim_dic[movie] += sim

            score = 0  #영화가 바뀌었으니 초기화한다
    
    for key in score_dic: 
        score_dic[key] = data[person]['avg'] + (score_dic[key] / sim_dic[key]) # 평점 총합/ 유사도 총합
        li.append((score_dic[key],key)) # list((tuple))의 리턴을 위해서.
    li.sort() #오름차순
    li.reverse() #내림차순
    return li

In [39]:
getRecommendation(ratings_expand, '소이현')

0.9661614876137535 홍수환
0.9438405255065262 최홍만
0.9437757473484876 설경구


[(3.91667234143915, '꾼'),
 (2.7989920841776077, '택시운전사'),
 (2.0343626946178484, '킹스맨:골든서클')]

* getRecommendation를 사용해서 최홍만 과 가장 유사한 사용자는?
* 단 유사도 함수를 sim_cosine, k는 2를 사용하시오

In [41]:
getRecommendation(ratings_expand, '최홍만', k=2, sim_function=sim_cosine)

0.9652129842456227 홍수환
0.9438405255065262 소이현


[(3.75, '택시운전사')]